In [1]:
import pandas as pd
pd.options.mode.chained_assignment = None  # default='warn'
import re
import requests
from bs4 import BeautifulSoup

In [74]:
#Reimportando arquivo - caso execucao tenha sido pausada
path_to_file = 'ListagemExport_teste00.csv'

listagem = pd.read_csv(path_to_file,
                       usecols= ['Projeto','Ano','Tipo','Ementa','Situacao','Autor','Codigo_Tipo'
                                 ,'Id_Projeto_Camara','Data_Projeto','Tramitacao_Encerrada','Aprovado'])

#Substitundo valores
listagem['Aprovado'] = listagem['Aprovado'].fillna('')
listagem['Tramitacao_Encerrada'] = listagem['Tramitacao_Encerrada'].fillna('')
listagem['Data_Projeto'] = listagem['Data_Projeto'].fillna('')
listagem['Codigo_Tipo'] = listagem['Codigo_Tipo'].fillna('')

listagem['Id_Projeto_Camara'] = listagem['Id_Projeto_Camara'].fillna(0)
listagem['Id_Projeto_Camara'] = listagem['Id_Projeto_Camara'].astype(int)

In [77]:
display(listagem)

,Projeto,Ano,Tipo,Ementa,Situacao,Autor,Codigo_Tipo,Id_Projeto_Camara,Data_Projeto,Tramitacao_Encerrada,Aprovado
0,11,2021,Requerimento,Requeiro a Excelentíssima Senhora Prefeita Mun...,Aprovado,Bejani Júnior,REQ,89111,,,
1,11,2021,Projeto de Resolução,Aprova as Contas da Prefeitura Municipal de Ju...,Aprovado,"André Luiz, Vagner de Oliveira, Nilton Militão",PRES,103039,,,


In [76]:

for i in listagem.index:
    if (listagem['Id_Projeto_Camara'][i] == '' or listagem['Id_Projeto_Camara'][i] == 0) and listagem['Codigo_Tipo'][i] not in ('DESCONHECIDO','EMENDA'): 
        with requests.Session() as session:
            proposicao = False

            tipo = ''
            ementa = ''
            situacao = ''

            id_projeto = ''
            id_projeto_camara = 0
            failed = False

            # Variaveis de requisicao
            num_proj = listagem['Projeto'][i]
            ano_proj = listagem['Ano'][i]
            tipo_proj = listagem['Codigo_Tipo'][i]

            # Variaveis de resposta
            #status = ''

            try:
                # Requisicao para o portal de propostas da Camara
                query = {"numero": num_proj, "ano": ano_proj, "codtipo": tipo_proj}
                resposta = requests.get('https://www.camarajf.mg.gov.br/sal/listapropos.php?', params=query)

                resp_format = BeautifulSoup(resposta.content, "html.parser")

                fnd_proposicoes = resp_format.find('table', 
                                                       {'border':'0', 
                                                        'cellpadding':'0', 
                                                        'cellspacing':'0', 
                                                        'height':'100%', 
                                                        'width':'100%'}).table.tr.td.find_all('a', 
                                                                                              {'target':'blank'})


                v_resultado = []
                m_proposicoes = [[None]*6]*(int(len(fnd_proposicoes)/6))


                for f in fnd_proposicoes:
                    v_resultado.append(f)

                l, c, a = 0, 0, 0

                while a < len(v_resultado):
                    m_proposicoes[l][c] = v_resultado[a]

                    a+=1
                        #print(m_proposicoes[1][2])

                    # Testes - percorrendo os calores de linhas, colunas e contador
                    #print((a%6))
                    #print(f'a = {a}')
                    #print(f'l = {l}')
                    #print(f'c = {c}')



                    if ((a%6)) == 0:

                        tipo = str(re.search('>(.+?)<', (str(m_proposicoes[l][2]))).group(1))
                        ementa = str(re.search('block;">(.+?)<', (str(m_proposicoes[l][4]))).group(1))
                        situacao = str(re.search('>(.+?)<', (str(m_proposicoes[l][5]))).group(1))

                        if tipo == listagem['Tipo'][i] and ementa == listagem['Ementa'][i]:
        #                if tipo == 'Pedido de Informação' and ementa == 'Solicita informações sobre os profissionais não docentes das instituições de Educação Infantil, públicas e privadas.':
                            id_projeto_camara = re.search('.PHP(.+?)TAR', str(m_proposicoes[0][0]).upper().replace(' ', '').replace('?NUM=','').replace('"','')).group(1)
                            situacao_prop = situacao
                            proposicao = True

                        l += 1
                        c = 0
                    else:
                        c += 1

                    if proposicao:
                        break
                   # print((m_proposicoes))

                    #for i in m_proposicoes:
                     #   print(m_proposicoes[i][2])


            except:
                print(resposta.status_code)

        session.close()
        listagem['Id_Projeto_Camara'][i] = id_projeto_camara 
        listagem['Situacao'][i] = situacao_proposicao

#tipo = re.search('>(.+?)<', (str(m_proposicoes[0][2]))).group(1)
print(situacao)
print(tipo) 
print(ementa)

                    
print(id_projeto_camara)


Transformado em Norma Jurídica
Projeto de Resolução
Aprova as Contas da Prefeitura Municipal de Juiz de Fora referentes ao exercício financeiro de 2017.
103039
